In [ ]:
## Imports
import ee
from google.colab import auth
import tensorflow as tf
import folium
from google.cloud import storage
from tqdm import tqdm
import pandas as pd
from copy import copy
import time
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import box
import json

## Install and import geopandas
!pip install geopandas
import geopandas as gpd

In [ ]:
## Authentication cell

# Authenticate Google account
auth.authenticate_user()

# # Trigger the authentication flow.
# ee.Authenticate()

## Another way to authenticate: go to https://console.cloud.google.com/iam-admin/serviceaccounts/details/
## add another key, download and upload the JSON file, then point to it below in ee.ServiceAccountCredentials,

credential_str = ' '
service_account = ' '
credentials = ee.ServiceAccountCredentials(service_account, credential_str)
ee.Initialize(credentials)

# Define bucket
bucket_name = ' '

# Define UTM CRS
utm_crs = 'EPSG:4326'

# Prepare Imagery

In [ ]:
geometry = ee.FeatureCollection("FAO/GAUL/2015/level0") \
              .filter(ee.Filter.eq('ADM0_NAME','Nigeria'))


imageCollection = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(geometry)

/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:204: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)


In [ ]:
#Sentinel Cloud Masking Function
def maskCloudAndShadowsSR(image):
  cloudProb = image.select('MSK_CLDPRB')
  snowProb = image.select('MSK_SNWPRB')
  cloud = cloudProb.lt(5)
  snow = snowProb.lt(5)
  scl = image.select('SCL')
  shadow = scl.eq(3); # 3 = cloud shadow
  cirrus = scl.eq(10); # 10 = cirrus
  # Cloud probability less than 5% or cloud shadow classification
  mask = (cloud.And(snow)).And(cirrus.neq(1)).And(shadow.neq(1))
  return image.updateMask(mask) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])

In [ ]:
# create list of size n for number of desired scenes
# 36 scenes per year: 10 days per scene
dayCount = ee.List.sequence(0, 363, 10.1)

def func_floor(x):
  return ee.Number(x).floor()

dayCount = dayCount.map(func_floor)

# run through the image collection and generate monthly median images

def func_composite(m):
  #set start date

  startMonth = ee.Number(1)
  startYear = ee.Number(2023)
  startDay = ee.Number(1)

  startDate = ee.Date.fromYMD(startYear, startMonth, startDay).advance(m, 'day')
  #set end date to 10 days after start date
  endDate = startDate.advance(10, 'day')
  #filter collection to images between start and end date
  filtered = imageCollection.filterDate(startDate, endDate)

  # mask for clouds and then take the monthly median composite
  composite = filtered.map(maskCloudAndShadowsSR).median()
  return composite \
      .set('month', startDate) \
      .set('system:time_start', startDate.millis())

composites = ee.ImageCollection.fromImages(dayCount.map(func_composite))

In [ ]:
#Add EVI band to each image in the collection
def addEVI(image):

  image = image.float()
  evi = image.expression('2.5 * ((NIR - RED) / (1 + NIR + 6 * RED - 7.5 * BLUE))',
    {'NIR': image.select('B8').divide(10000),
     'RED': image.select('B4').divide(10000),
     'BLUE': image.select('B2').divide(10000)})
  return image.addBands(evi)

In [ ]:
#rename EVI band
s2EVI = composites.map(addEVI);
s2EVI = s2EVI.select(['constant'],['EVI'])
composite = s2EVI.select('EVI').toBands().unmask(ee.Image.constant(0)).clip(geometry)


#### Load geometry for export

This will either be one large polygon that will be broken up into smaller components for large-scale imagery generation (inference), or it will be a series of training polygons collected via hand.

In [ ]:
# Define one booleans to govern whether s2 imagery should be generated over a
# inference AOI, or for polygons collected for training. If the polygons are for
# training, define another boolean for whether the labels contain irrigation.
import shapely

geom_for_inference = True
irrigated_labels = False


def get_squares_from_aoi(aoi, side_length=0.25):
    """
    Divide a AOI (Shapely MultiPolygon) into squares of equal area.
    AOI must be in EPSG:4326 crs.

    1 degree = 111km
    0.25 degree = 25km = 25000m = 2500 s2 pixels

    `side_length` : required side of square
    """
    rect_coords = np.array(aoi.boundary.coords.xy)
    y_list = rect_coords[1]
    x_list = rect_coords[0]
    y1 = min(y_list)
    y2 = max(y_list)
    x1 = min(x_list)
    x2 = max(x_list)
    width = x2 - x1
    height = y2 - y1


    xcells = int(np.round(width / side_length))
    ycells = int(np.round(height / side_length))

    geom_list = []
    ee_geom_list = []

    for x in range(xcells):
        for y in range(ycells):
            ee_cell =  ee.Geometry.Rectangle(
                    x*side_length + x1,
                    y*side_length + y1,
                    (x+1)*side_length + x1,
                    (y+1)*side_length + y1
            )

            cell = box(x*side_length + x1,
                       y*side_length + y1,
                       (x+1)*side_length + x1,
                       (y+1)*side_length + y1)

            ee_geom_list.append(ee_cell)
            geom_list.append(cell)

    centroids = [i.centroid for i in geom_list]

    gdf_dict = {'id':range(len(geom_list)),
                'side_length_deg': side_length,
                'centroids': centroids,
                'ee_geometry': ee_geom_list,
                'geometry':geom_list}
    gdf = gpd.GeoDataFrame(gdf_dict, crs='EPSG:4326')

    return gdf


if geom_for_inference:
    geometry_filename = f'gs://{bucket_name}/shapefiles/Zambiabb2.geojson'
    image_save_prefix = f'raw_imagery/imagery_for_inference/Zambia2'

    full_geom_gdf = gpd.read_file(geometry_filename).explode(index_parts=True)

    aoi = full_geom_gdf['geometry'].iloc[0]
    gdf = get_squares_from_aoi(aoi)
    gdf = gdf.to_crs(utm_crs)


else:
    if irrigated_labels:
        irrig_str = 'irrig'
    else:
        irrig_str = 'noirrig'

    # geometry_filename = f'gs://gee_irrigation_detection/shapefiles/tana_10k_{irrig_str}_2020.geojson'
    # image_save_prefix = f'raw_imagery/imagery_for_labels/tana_10k/{irrig_str}'

    geometry_filename = f'gs://gee_irrigation_detection/shapefiles/BF_{irrig_str}_HS.geojson'
    image_save_prefix = f'raw_imagery/imagery_for_labels/BF/{irrig_str}'

    gdf = gpd.read_file(geometry_filename)
    centroids = [i.centroid for i in gdf['geometry']]
    gdf['centroids'] = centroids

    # Extract geometries and convert to ee.Geometry format
    geom_json_list = [shapely.geometry.mapping(i) for i in
                      gdf['geometry']]
    print(geom_json_list)
    ee_geom_list = [ee.Geometry.Polygon(i["coordinates"]) for i in geom_json_list]
    gdf['ee_geometry'] = ee_geom_list

    gdf = gdf.to_crs(utm_crs)
    gdf['id'] = range(len(gdf))

In [ ]:
print(len(gdf))

25


In [ ]:
  ## Iterate through the geometries for uploading S2 composites


## Need to do first 3
for ix in tqdm(range(4,25)): ## Up to len(gdf)


    id = gdf['id'].iloc[ix]
    x_c = np.round(gdf['centroids'].iloc[ix].x, decimals=3)
    y_c = np.round(gdf['centroids'].iloc[ix].x, decimals=3)
    export_geometry = gdf['ee_geometry'].iloc[ix].transform(utm_crs, maxError=3)

    print(f'{image_save_prefix}/s2_timeseries_id_{id}_x_{x_c}_y_{y_c}')

    # # Setup the task.
    image_task = ee.batch.Export.image.toCloudStorage(
    image=composite,
    description='imageToCloudExample',
    fileNamePrefix=f'{image_save_prefix}/s2_timeseries_id_{id}_x_{x_c}_y_{y_c}',
    bucket=bucket_name,
    scale=10,
    # crs=utm_crs,
    region=export_geometry,
    fileFormat='GeoTIFF',
    formatOptions= {
        'cloudOptimized': True,
        }
    )
    ## Start task
    image_task.start()

    # Monitor the task.
    while image_task.status()['state'] in ['READY', 'RUNNING']:
        print(image_task.status())
        time.sleep(10)
    else:
        print(image_task.status())


  0%|          | 0/21 [00:00<?, ?it/s]

raw_imagery/imagery_for_inference/Zambia2/s2_timeseries_id_4_x_31.925_y_31.925
{'state': 'READY', 'description': 'imageToCloudExample', 'priority': 100, 'creation_timestamp_ms': 1719041138227, 'update_timestamp_ms': 1719041138227, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'SOGK2WT6HM3IGUBXKW4A6J5L', 'name': 'projects/earthengine-legacy/operations/SOGK2WT6HM3IGUBXKW4A6J5L'}
{'state': 'RUNNING', 'description': 'imageToCloudExample', 'priority': 100, 'creation_timestamp_ms': 1719041138227, 'update_timestamp_ms': 1719041142197, 'start_timestamp_ms': 1719041142160, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'SOGK2WT6HM3IGUBXKW4A6J5L', 'name': 'projects/earthengine-legacy/operations/SOGK2WT6HM3IGUBXKW4A6J5L'}
{'state': 'RUNNING', 'description': 'imageToCloudExample', 'priority': 100, 'creation_timestamp_ms': 1719041138227, 'update_timestamp_ms': 1719041142197, 'start_timestamp_ms': 1719041142160, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'SOGK2WT6HM3IGUBXKW4A6J

  5%|▍         | 1/21 [14:19<4:46:27, 859.39s/it]

{'state': 'COMPLETED', 'description': 'imageToCloudExample', 'priority': 100, 'creation_timestamp_ms': 1719041138227, 'update_timestamp_ms': 1719041990131, 'start_timestamp_ms': 1719041142160, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://console.developers.google.com/storage/browser/gee_irrigation_detection/raw_imagery/imagery_for_inference/Zambia2/'], 'attempt': 1, 'batch_eecu_usage_seconds': 2859.8046875, 'id': 'SOGK2WT6HM3IGUBXKW4A6J5L', 'name': 'projects/earthengine-legacy/operations/SOGK2WT6HM3IGUBXKW4A6J5L'}
raw_imagery/imagery_for_inference/Zambia2/s2_timeseries_id_5_x_32.175_y_32.175
{'state': 'READY', 'description': 'imageToCloudExample', 'priority': 100, 'creation_timestamp_ms': 1719041997456, 'update_timestamp_ms': 1719041997456, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'MBSG5YXSIQWOSNABJ74S3RTE', 'name': 'projects/earthengine-legacy/operations/MBSG5YXSIQWOSNABJ74S3RTE'}
{'state': 'RUNNING', 'description': 'imageToCloudExample', 'priority': 1

 10%|▉         | 2/21 [28:51<4:34:35, 867.14s/it]

...

{'state': 'COMPLETED', 'description': 'imageToCloudExample', 'priority': 100, 'creation_timestamp_ms': 1719056233124, 'update_timestamp_ms': 1719056949305, 'start_timestamp_ms': 1719056235113, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://console.developers.google.com/storage/browser/gee_irrigation_detection/raw_imagery/imagery_for_inference/Zambia2/'], 'attempt': 1, 'batch_eecu_usage_seconds': 3231.000244140625, 'id': 'IJ4YWOOL7TRTZXNPBSGFHRMS', 'name': 'projects/earthengine-legacy/operations/IJ4YWOOL7TRTZXNPBSGFHRMS'}
raw_imagery/imagery_for_inference/Zambia2/s2_timeseries_id_23_x_32.925_y_32.925
{'state': 'READY', 'description': 'imageToCloudExample', 'priority': 100, 'creation_timestamp_ms': 1719056950955, 'update_timestamp_ms': 1719056950955, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'Y3SXXAJGGBB74GWRTR4LNOTC', 'name': 'projects/earthengine-legacy/operations/Y3SXXAJGGBB74GWRTR4LNOTC'}
{'state': 'RUNNING', 'description': 'imageToCloudExample', 'priori

 95%|█████████▌| 20/21 [4:41:19<14:13, 853.53s/it]

{'state': 'COMPLETED', 'description': 'imageToCloudExample', 'priority': 100, 'creation_timestamp_ms': 1719056950955, 'update_timestamp_ms': 1719058014558, 'start_timestamp_ms': 1719056953515, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://console.developers.google.com/storage/browser/gee_irrigation_detection/raw_imagery/imagery_for_inference/Zambia2/'], 'attempt': 1, 'batch_eecu_usage_seconds': 3368.522705078125, 'id': 'Y3SXXAJGGBB74GWRTR4LNOTC', 'name': 'projects/earthengine-legacy/operations/Y3SXXAJGGBB74GWRTR4LNOTC'}
raw_imagery/imagery_for_inference/Zambia2/s2_timeseries_id_24_x_32.925_y_32.925
{'state': 'READY', 'description': 'imageToCloudExample', 'priority': 100, 'creation_timestamp_ms': 1719058017434, 'update_timestamp_ms': 1719058017434, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'VRLQTUBAB4OC3OUFE5P5AROG', 'name': 'projects/earthengine-legacy/operations/VRLQTUBAB4OC3OUFE5P5AROG'}
{'state': 'RUNNING', 'description': 'imageToCloudExample', 'priori

100%|██████████| 21/21 [4:50:05<00:00, 828.82s/it]

{'state': 'COMPLETED', 'description': 'imageToCloudExample', 'priority': 100, 'creation_timestamp_ms': 1719058017434, 'update_timestamp_ms': 1719058541418, 'start_timestamp_ms': 1719058019517, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://console.developers.google.com/storage/browser/gee_irrigation_detection/raw_imagery/imagery_for_inference/Zambia2/'], 'attempt': 1, 'batch_eecu_usage_seconds': 3354.611572265625, 'id': 'VRLQTUBAB4OC3OUFE5P5AROG', 'name': 'projects/earthengine-legacy/operations/VRLQTUBAB4OC3OUFE5P5AROG'}


In [ ]:
## Not sure how to cancel operations
# ee.data.listOperations()
# ee.data.cancelOperation('projects/earthengine-legacy/operations/RFES7VXEEFBYDF33E4CNGMRS')